In [ ]:
# SPDX-License-Identifier: Apache-2.0 AND CC-BY-NC-4.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<img src="./images/nvmath_head_panel@0.5x.png" alt="nvmath-python" />

# Getting Started with nvmath-python: Memory and Execution Spaces



## Exercise: Estimate CPU-GPU Data Transfer Overhead

We see a non-negligible performance difference between data residing in CPU memory space versus GPU memory space in the above example. Given that the execution space is always GPU, estimate the data transfer cost. Implement a dedicated benchmark for a cross-check.

**Hint**: You can use CuPy to explicitly time the data transfer between CPU and GPU using `cp.asarray()` and `cp.asnumpy()` operations.

In [ ]:
## Exercise: Evaluate performance of CuPy `@`-based vs. `matmul`-based implementations of GEMM

import nvmath
import numpy as np
import cupy as cp
from cupyx.profiler import benchmark

# Define matmul parameters
m, n, k = 8000, 2000, 4000
a_cpu = np.random.randn(m, k).astype(np.float32)
b_cpu = np.random.randn(k, n).astype(np.float32)

a_gpu = cp.random.randn(m, k, dtype=cp.float32)
b_gpu = cp.random.randn(k, n, dtype=cp.float32)

t_cpu_gpu_transfer_and_gpu_compute = benchmark(lambda: nvmath.linalg.advanced.matmul(a_cpu, b_cpu), n_repeat=5, n_warmup=1).gpu_times.min()
print(f"Time taken for CPU -> GPU -> matmul -> CPU: {t_cpu_gpu_transfer_and_gpu_compute:.4f} seconds")

t_gpu_compute = benchmark(lambda: nvmath.linalg.advanced.matmul(a_gpu, b_gpu), n_repeat=5, n_warmup=1).gpu_times.min()
print(f"Time taken for GPU -> matmul -> GPU: {t_gpu_compute:.4f} seconds")

t_cpu_gpu_transfer_estimate = t_cpu_gpu_transfer_and_gpu_compute - t_gpu_compute
print(f"Estimated time taken for CPU -> GPU -> CPU transfer): {t_cpu_gpu_transfer_estimate:.4f} seconds")

t_cpu_gpu_transfer = benchmark(lambda: cp.asarray(a_cpu), n_repeat=5, n_warmup=1).gpu_times.min()
print(f"Time taken for CPU -> GPU transfer: {t_cpu_gpu_transfer:.4f} seconds")

t_gpu_cpu_transfer = benchmark(lambda: cp.asnumpy(a_gpu), n_repeat=5, n_warmup=1).gpu_times.min()
print(f"Time taken for GPU -> CPU transfer: {t_gpu_cpu_transfer:.4f} seconds")

t_cpu_gpu_cpu_transfer = t_cpu_gpu_transfer + t_gpu_cpu_transfer
print(f"Time taken for CPU -> GPU and GPU -> CPU transfers: {t_cpu_gpu_cpu_transfer:.4f} seconds")


Note that we spent more time on data transfers between CPU and GPU (and back) than on doing the compute. Whenever possible avoid data transfers between memory spaces unless the amount of compute exceeds far the time spent in data transfers.